In [ ]:

import numpy as np
np.set_printoptions(suppress=True, precision=4)

def set_lowVal_zero(X):
    low_values_indices = abs(X) < 9e-15   # where values are low
    X[low_values_indices] = 0             # all low values set to 0
    return X

def Householder(x, i):
    alpha = -np.sign(x[i]) * np.linalg.norm(x)
    e = np.zeros(len(x)); e[i] = 1.0
    
    v = (x - alpha * e)
    w = v / np.linalg.norm(v)
    P = np.eye(len(x)) - 2 * np.outer(w, w.T)
    
    return P

def Golub_Kahan(X):
    m, n = X.shape
    J = X.copy()

    U = np.identity(m)
    V = np.identity(n)

    for i in range(n - 2):  # n-2 steps
        # --- Column reflector (left multiply)
        h = np.zeros(m)
        h[i:] = J[i:, i]
        P = Householder(h, i)
        J = set_lowVal_zero(P @ J)
        U = U @ P

        # --- Row reflector (right multiply)
        h = np.zeros(n)
        h[i+1:] = J[i, i+1:]
        Q = Householder(h, i+1)
        J = set_lowVal_zero(J @ Q)
        V = V @ Q

    return U, J, V


def givensrotation_R(a, b):
    if b == 0:
        return 1.0, 0.0
    r = np.hypot(a, b)
    c = a / r
    s = -b / r
    return c, s

def QR_givens(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m,dtype=float)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation_R(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(np.conj(Q)), R

def QR_iterate(A):
    n = A.shape[0]
    Q_total = np.eye(n)
    A_k = A.copy()
    for _ in range(100):
        Q, R = QR_givens(A_k)
        A_k = R @ Q
        Q_total = Q_total @ Q
    
    
        
    return Q_total, A_k

def eigenvalues(A):
    
    Q, R = QR_iterate(A)
    eigenvals = np.diag(R)
    eigenvecs = Q
    return eigenvals, eigenvecs

def fix_sign(v):
    original_shape = v.shape
    v_flat = v.flatten()
    idx = np.argmax(np.abs(v_flat))
    if v_flat[idx] < 0:
        v_flat = -v_flat
    return v_flat.reshape(original_shape)

def compute_svd(A):
    m, n = A.shape
    AAt = A @ A.T
    AtA = A.T @ A

    vals1, vecs1 = eigenvalues(AAt)
    vals2, vecs2 = eigenvalues(AtA)
    #vecs1 = fix_sign(vecs1)
    #vecs2 = fix_sign(vecs2)
    
    sigma = np.sqrt(np.clip(vals1, 0, None))
    V = vecs2

    U = vecs1

    Sigma = np.zeros((m, n))
    np.fill_diagonal(Sigma, sigma)
    
    print(vals1)
    print(vals2)
    
    U[np.abs(U) < 1e-4] = 0
    Sigma[np.abs(Sigma) < 1e-4] = 0
    V[np.abs(V) < 1e-4] = 0
    
    for i in range(min(m,n)):
        Av = A @ V[:, i]
        sig_ui = sigma[i] * U[:, i]
        
        if np.dot(Av, sig_ui) < 0: 
            U[:, i] *= -1
            #V[:, i] *= -1

    return U, Sigma, V.T

def pinv_from_svd(U, Sigma, Vt, tol=1e-10):
    m, n = Sigma.shape
    sigma_vals = np.diag(Sigma)[:min(m, n)]  

    sigma_inv = np.array([1/s if s > tol else 0 for s in sigma_vals])
    Sigma_inv = np.zeros((n, m))  
    for i in range(len(sigma_inv)):
        Sigma_inv[i, i] = sigma_inv[i]

    # A⁺ = V @ Sigma⁺ @ U.T
    return Vt.T @ Sigma_inv @ U.T

In [8]:
def pinv(A):
    u,j,v = Golub_Kahan(A)
    print(np.allclose(A, u @ j @ v.T))
    U, Sigma, V = compute_svd(j)
    print(np.allclose(U@Sigma@V,j))
    U_final = u @ U
    V_final = v.conj().T @ V.conj().T
    print(np.allclose(A, U_final @ Sigma @ V_final.T))
    return pinv_from_svd(U_final, Sigma, V_final.T)

In [21]:
A = np.random.randn(50,2)

In [22]:
A

array([[ 1.1908,  0.4558],
       [ 1.0411,  2.1457],
       [ 0.577 , -0.4101],
       [ 1.3603,  0.8675],
       [ 1.4915, -1.2719],
       [-0.0658,  2.7518],
       [-0.8869,  0.5693],
       [ 0.8156, -0.5061],
       [-0.4606,  0.3516],
       [ 1.2035, -0.6503],
       [ 0.7424, -0.2699],
       [ 1.1389, -0.0523],
       [-0.1987, -0.1239],
       [-0.7017, -0.5434],
       [-0.1242,  0.8381],
       [ 0.647 , -1.593 ],
       [ 1.9086,  1.4331],
       [-1.2191,  0.0995],
       [ 0.8871, -0.4589],
       [-1.3779,  0.2237],
       [ 0.6181,  0.64  ],
       [-0.653 , -2.0002],
       [ 0.7931,  1.5508],
       [ 0.468 ,  0.2311],
       [ 1.7379,  0.4536],
       [-0.1065, -0.9949],
       [-1.073 , -1.9186],
       [ 1.176 , -0.1564],
       [-0.906 , -0.165 ],
       [-0.4542,  0.3613],
       [-1.6034,  1.8564],
       [-0.3919, -0.6681],
       [ 1.5289, -2.2593],
       [-0.6949, -1.0574],
       [-0.3288,  0.1694],
       [-1.8627, -0.4656],
       [ 1.5416,  0.2865],
 

In [23]:
pinv(A)

True
[63.504  50.7647  0.     -0.     -0.     -0.      0.      0.     -0.
 -0.      0.      0.      0.     -0.     -0.      0.      0.     -0.
 -0.      0.      0.      0.     -0.     -0.      0.      0.     -0.
 -0.     -0.      0.      0.     -0.     -0.     -0.      0.      0.
 -0.     -0.      0.      0.     -0.      0.     -0.      0.      0.
 -0.      0.      0.      0.      0.    ]
[63.504  50.7647]
True
True


array([[ 0.0197,  0.0192,  0.0088,  0.0229,  0.0226,  0.0021, -0.0136,
         0.0126, -0.007 ,  0.0186,  0.0116,  0.0183, -0.0033, -0.0119,
        -0.0011,  0.0086,  0.0324, -0.0195,  0.0138, -0.0219,  0.0107,
        -0.0128,  0.0145,  0.0078,  0.0285, -0.0028, -0.0195,  0.0188,
        -0.0148, -0.0069, -0.0237, -0.0071,  0.0221, -0.0124, -0.0051,
        -0.0305,  0.0252, -0.0143,  0.0144,  0.0111, -0.0212,  0.0165,
        -0.0183, -0.0015, -0.0589, -0.0137,  0.0101,  0.0049, -0.0092,
        -0.0053],
       [ 0.0102,  0.0427, -0.0073,  0.0183, -0.0229,  0.0532,  0.01  ,
        -0.0089,  0.0063, -0.0112, -0.0044,  0.0003, -0.0026, -0.0113,
         0.0161, -0.0301,  0.0299,  0.0005, -0.0079,  0.0028,  0.0131,
        -0.0394,  0.0309,  0.005 ,  0.0107, -0.0194, -0.0383, -0.0017,
        -0.0042,  0.0065,  0.0341, -0.0134, -0.042 , -0.0212,  0.0029,
        -0.0111,  0.0073,  0.0075,  0.007 , -0.0032,  0.0191,  0.0072,
        -0.0182,  0.0049,  0.0132,  0.0318,  0.0116, -0.002

In [24]:
np.linalg.pinv(A)

array([[ 0.0197,  0.0192,  0.0088,  0.0229,  0.0226,  0.0021, -0.0136,
         0.0126, -0.007 ,  0.0186,  0.0116,  0.0183, -0.0033, -0.0119,
        -0.0011,  0.0086,  0.0324, -0.0195,  0.0138, -0.0219,  0.0107,
        -0.0128,  0.0145,  0.0078,  0.0285, -0.0028, -0.0195,  0.0188,
        -0.0148, -0.0069, -0.0237, -0.0071,  0.0221, -0.0124, -0.0051,
        -0.0305,  0.0252, -0.0143,  0.0144,  0.0111, -0.0212,  0.0165,
        -0.0183, -0.0015, -0.0589, -0.0137,  0.0101,  0.0049, -0.0092,
        -0.0053],
       [ 0.0102,  0.0427, -0.0073,  0.0183, -0.0229,  0.0532,  0.01  ,
        -0.0089,  0.0063, -0.0112, -0.0044,  0.0003, -0.0026, -0.0113,
         0.0161, -0.0301,  0.0299,  0.0005, -0.0079,  0.0028,  0.0131,
        -0.0394,  0.0309,  0.005 ,  0.0107, -0.0194, -0.0383, -0.0017,
        -0.0042,  0.0065,  0.0341, -0.0134, -0.042 , -0.0212,  0.0029,
        -0.0111,  0.0073,  0.0075,  0.007 , -0.0032,  0.0191,  0.0072,
        -0.0182,  0.0049,  0.0132,  0.0318,  0.0116, -0.002

In [25]:
np.allclose(pinv(A), np.linalg.pinv(A))  

True
[63.504  50.7647  0.     -0.     -0.     -0.      0.      0.     -0.
 -0.      0.      0.      0.     -0.     -0.      0.      0.     -0.
 -0.      0.      0.      0.     -0.     -0.      0.      0.     -0.
 -0.     -0.      0.      0.     -0.     -0.     -0.      0.      0.
 -0.     -0.      0.      0.     -0.      0.     -0.      0.      0.
 -0.      0.      0.      0.      0.    ]
[63.504  50.7647]
True
True


True